This notebook illustrates [DALL·E mini](https://github.com/borisdayma/dalle-mini) inference pipeline.

For more understanding of the model, refer to [the report](https://wandb.ai/dalle-mini/dalle-mini/reports/DALL-E-mini--Vmlldzo4NjIxODA).

<img src="https://github.com/borisdayma/dalle-mini/blob/main/img/logo.png?raw=true" width="100">

In [ ]:
# Text Prompt
text_input = 'robot friend by Giuseppe Arcimboldo'  #@param {type: "string"}

# Perform 4x neural super-resolution (from 256x256px to 1024x124)
super_resolution = True   #@param {type: "boolean"}

output_path = '/content/output'

## Install dependencies

In [ ]:
%cd /contnet
!pip install -q transformers flax
!pip install -q git+https://github.com/patil-suraj/vqgan-jax.git  # VQGAN model in JAX
!git clone https://github.com/borisdayma/dalle-mini  # Model files
%cd /content/dalle-mini/

## Generate encoded images

We generate prediction samples from a text prompt using `flax-community/dalle-mini` model.

In [ ]:
from dalle_mini.model import CustomFlaxBartForConditionalGeneration
from transformers import BartTokenizer
import jax
import random
from tqdm.notebook import tqdm, trange

In [ ]:
# make sure we use compatible versions
DALLE_REPO = 'flax-community/dalle-mini'
DALLE_COMMIT_ID = '4d34126d0df8bc4a692ae933e3b902a1fa8b6114'

In [ ]:
# set up tokenizer and model
tokenizer = BartTokenizer.from_pretrained(DALLE_REPO, revision=DALLE_COMMIT_ID)
model = CustomFlaxBartForConditionalGeneration.from_pretrained(DALLE_REPO, revision=DALLE_COMMIT_ID)

In [ ]:
# set a prompt
prompt = text_input

In [ ]:
# tokenize the prompt
tokenized_prompt = tokenizer(prompt, return_tensors='jax', padding='max_length', truncation=True, max_length=128)
tokenized_prompt

Notes:

* `0`: BOS, special token representing the beginning of a sequence
* `2`: EOS, special token representing the end of a sequence
* `1`: special token representing the padding of a sequence when requesting a specific length

In [ ]:
n_predictions = 8

# create random keys
seed = random.randint(0, 2**32-1)
key = jax.random.PRNGKey(seed)
subkeys = jax.random.split(key, num=n_predictions)
subkeys

In [ ]:
# generate sample predictions
encoded_images = [model.generate(**tokenized_prompt, do_sample=True, num_beams=1, prng_key=subkey) for subkey in tqdm(subkeys)]
encoded_images[0]

The first token (`16384`) is a special token representing the start of a sequence in the decoder (not part of the image codebook).

In [ ]:
# remove first token (BOS)
encoded_images = [img.sequences[..., 1:] for img in encoded_images]
encoded_images[0]

The generated images are now represented by 256 tokens.

In [ ]:
encoded_images[0].shape

## Decode images

The generated images need to be decoded with `flax-community/vqgan_f16_16384`.

In [ ]:
from vqgan_jax.modeling_flax_vqgan import VQModel
import numpy as np
from PIL import Image

In [ ]:
# make sure we use compatible versions
VQGAN_REPO = 'flax-community/vqgan_f16_16384'
VQGAN_COMMIT_ID = '90cc46addd2dd8f5be21586a9a23e1b95aa506a9'

In [ ]:
# set up VQGAN
vqgan = VQModel.from_pretrained(VQGAN_REPO, revision=VQGAN_COMMIT_ID)

In [ ]:
# decode images
decoded_images = [vqgan.decode_code(encoded_image) for encoded_image in tqdm(encoded_images)]
decoded_images[0]

In [ ]:
# normalize images
clipped_images = [img.squeeze().clip(0., 1.) for img in decoded_images]

In [ ]:
# convert to image
images = [Image.fromarray(np.asarray(img * 255, dtype=np.uint8)) for img in clipped_images]

In [ ]:
del model
del vqgan
import gc
gc.collect()

In [ ]:
#@title Upscale images/video frames


loaded_upscale_model = False

def upscale(filepath):
  if not super_resolution:
    return
  global loaded_upscale_model
  if not loaded_upscale_model:
    # Clone Real-ESRGAN and enter the Real-ESRGAN
    !git clone https://github.com/xinntao/Real-ESRGAN.git
    %cd /content/Real-ESRGAN
    # Set up the environment
    !pip install basicsr
    !pip install facexlib
    !pip install gfpgan
    !pip install -r requirements.txt
    !python setup.py develop
    # Download the pre-trained model
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
    %cd -
    loaded_upscale_model = True 
  
  %cd /content/Real-ESRGAN
  !python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input $filepath --netscale 4 --outscale 3.5 --half --output $output_path
  filepath_out = filepath.replace(".jpg","_out.jpg")
  !mv -v $filepath_out $filepath
  %cd -

In [ ]:
!mkdir -p $output_path
# display an image
for idx in range(len(images)):
    #display(images[idx])
    idx_str = '{:05}'.format(idx)
    save_path = f"{output_path}/image_{idx}.png"
    images[idx].save(save_path)
    print("saved image to ", save_path)
    upscale(save_path)

In [ ]:
out_file=output_path+"/video.mp4"

!ffmpeg -r 0.5 -i $output_path/%*.png -y -c:v libx264 -filter:v fps=fps=20 /tmp/vid_no_audio.mp4
!ffmpeg -i /tmp/vid_no_audio.mp4 -f lavfi -i anullsrc -c:v copy -c:a aac -shortest -y "$out_file"

print("Written", out_file)
!sleep 2
!rm -r /tmp/ffmpeg
!sleep 5